# 1D卷积  ---等效于CNN做文本分类

![](http://op4a94iq8.bkt.clouddn.com/18-7-13/49813055.jpg)

In [27]:
# ------------------------------------------------------------
import torch 
import torch.nn as nn
import torch.nn.functional as F

m = nn.Conv1d(2, 3, 4, stride=2,padding=0,groups=1,dilation=1)

input = torch.randn(5, 2, 10)
print("input size:",input.shape)

output = m(input)
print("output size:",output.shape)

print("filter size:",m.weight.shape)
print(m.weight.type)

input size: torch.Size([5, 2, 10])
output size: torch.Size([5, 3, 4])
filter size: torch.Size([3, 2, 4])
<built-in method type of Parameter object at 0x7f8d25ff5990>


In [48]:

# pool of non-square window
input = torch.randn(1, 512, 14, 14) # batch_size=1
print("input size:",input.shape)
print(input.type)

m = nn.AvgPool2d(14)

output = m(input)
print("output size:",output.shape)

avg=output.view(output.shape[0],1,output.shape[1])
print("avg size:",avg.shape)

one_conv_filter = nn.Conv1d(1, 1024, 3, stride=1,padding=1,groups=1,dilation=1)

out_1 = one_conv_filter(avg)
print("out_1 size:",out_1.shape)
print("filter size:",one_conv_filter.weight.shape)


# --------------------- 加权融合 -----------------------



input size: torch.Size([1, 512, 14, 14])
<built-in method type of Tensor object at 0x7f8d2612c1b0>
output size: torch.Size([1, 512, 1, 1])
avg size: torch.Size([1, 1, 512])
out_1 size: torch.Size([1, 1024, 512])
filter size: torch.Size([1024, 1, 3])


In [85]:
# 不考虑batchsize时实现了1x1卷积
y=torch.Tensor(
    [
       [[[0.1]],[[0.7]],[[3]],],
       [[[0.2]],[[0.4]],[[10]],]
    ])

print("Y size:",y.shape)

x=torch.Tensor(
    [
        [[1,1,],[1,1,],],
        
        [[2,2,],[2,2,],],
        
        [[3,3,],[3,3,],],
    ])

print("X size:",x.shape)

out=x*y
print(out.shape)
print(out[1]) 

# ----------------- 聚合 ----------
res=out.sum(dim=1)
print(res.shape)
print(res)

Y size: torch.Size([2, 3, 1, 1])
X size: torch.Size([3, 2, 2])
torch.Size([2, 3, 2, 2])
tensor([[[  0.2000,   0.2000],
         [  0.2000,   0.2000]],

        [[  0.8000,   0.8000],
         [  0.8000,   0.8000]],

        [[ 30.0000,  30.0000],
         [ 30.0000,  30.0000]]])
torch.Size([2, 2, 2])
tensor([[[ 10.5000,  10.5000],
         [ 10.5000,  10.5000]],

        [[ 31.0000,  31.0000],
         [ 31.0000,  31.0000]]])


In [66]:
y=torch.Tensor(
    [
       [[[0.1]],[[0.03]],[[4]]],
       [[[20]],[[0.2]],[[0.6]]],
    ])

print(y.shape)

x=torch.Tensor(
    [
        [[1,1,],[1,1,],],
        
        [[2,2,],[2,2,],],
        
        [[3,3,],[3,3,],],
    ])

print(x.shape)

out=x*y
print(out.shape)
print(out)

torch.Size([2, 3, 1, 1])
torch.Size([3, 2, 2])
torch.Size([2, 3, 2, 2])
tensor([[[[  0.1000,   0.1000],
          [  0.1000,   0.1000]],

         [[  0.0600,   0.0600],
          [  0.0600,   0.0600]],

         [[ 12.0000,  12.0000],
          [ 12.0000,  12.0000]]],


        [[[ 20.0000,  20.0000],
          [ 20.0000,  20.0000]],

         [[  0.4000,   0.4000],
          [  0.4000,   0.4000]],

         [[  1.8000,   1.8000],
          [  1.8000,   1.8000]]]])


In [ ]:
# 实验一下 mobienet中的机制添加
import torch 
import torch.nn as nn
import torch.nn.functional as F

m = nn.Conv1d(2, 3, 4, stride=2,padding=0,groups=1,dilation=1)

# 经典mobilenet输入
input = torch.randn(64, 14, 14,512)
avg=nn.AvgPool2d()


print("input size:",input.shape)

output = m(input)
print("output size:",output.shape)

print("filter size:",m.weight.shape)

In [1]:
'''MobileNet in PyTorch.
See the paper "MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications"
for more details.
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def test():
    net = MobileNet()
    x = torch.randn(1,3,32,32)
    y = net(x)
    print(y.size())

# test()